In [ ]:
import pandas as pd
import shortuuid

In [ ]:
df = pd.read_csv('dataset/cleaned_nur_data.csv', index_col=0)
df.head()

In [ ]:
def generate_short_unique_id(row):
    concat_str = str(row['rank']) + str(row['column2']) + str(row['column3'])
    # Generate a UUID from the concatenated string
    unique_id = shortuuid.uuid(name=concat_str)
    return unique_id

In [ ]:
df_nur = df.copy()
df_nur = df_nur.rename(columns={'state': 'state_id'})
df_nur['rank_id'] = 

In [ ]:
import pandas as pd
import shortuuid

# Sample DataFrame
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C'],
    'column3': ['X', 'Y', 'Z']
}
df = pd.DataFrame(data)

# Function to generate a short unique ID using shortuuid
def generate_short_unique_id(row):
    # Concatenate the values from multiple columns into a string
    concat_str = str(row['column1']) + str(row['column2']) + str(row['column3'])
    # Generate a UUID from the concatenated string
    unique_id = shortuuid.uuid(name=concat_str)
    return unique_id

# Apply the function to each row and create a new column 'short_unique_id'
df['short_unique_id'] = df.apply(generate_short_unique_id, axis=1)

print(df)